In [5]:
    import pymongo

In [6]:
client=pymongo.MongoClient('localhost',27017)

In [7]:
db=client['chapter3']

In [8]:
collection=db['income']

In [9]:
with open('income_header.txt') as f_in:
    column_headings=f_in.readline()

In [10]:
column_headings

'age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label'

In [11]:
column_headings_list=column_headings.split(',')

In [12]:
column_headings_list

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [14]:
#cleaning 
row_dict_list=list()
with open('income.txt') as f_in:
    for line in  f_in:
        row_list=line.rstrip('\n').split(',')
        row_dict=dict(zip(column_headings_list,row_list))
        row_dict_list.append(row_dict)
        collection.insert_one(row_dict)#put it in database
        

In [15]:
collection.count()

65124

In [16]:
#pperform quality checks
age39=collection.find_one({'age':39})

In [22]:
collection.delete_many({})

In [23]:
collection.count()

0

In [24]:
#converting age to integer
with open('income.txt') as f_in:
    for line in  f_in:
        row_list=line.rstrip('\n').split(',')
        row_dict=dict(zip(column_headings_list,row_list))
        try:
            row_dict['age']=int(row_dict['age'])
            row_dict_list.append(row_dict)
            collection.insert_one(row_dict)#put it in database
        except:
            pass

In [26]:
over_35=collection.find({'age':{"$gt":35}})

In [27]:
type(over_35)

pymongo.cursor.Cursor

In [29]:
over_35.next()

{u'_id': ObjectId('594209576a63a64d44036645'),
 u'age': 39,
 u'capital-gain': u' 2174',
 u'capital-loss': u' 0',
 u'education': u' Bachelors',
 u'education-num': u' 13',
 u'fnlwgt': u' 77516',
 u'hours-per-week': u' 40',
 u'label': u' <=50K',
 u'marital-status': u' Never-married',
 u'native-country': u' United-States',
 u'occupation': u' Adm-clerical',
 u'race': u' White',
 u'relationship': u' Not-in-family',
 u'sex': u' Male',
 u'workclass': u' State-gov'}

In [30]:
over_35.count()

17636

In [48]:
import time
start=time.time()
#age50=collection.find({'age':{'$eq':50}})
over_35=collection.find({'age':{"$gt":55}})
end=time.time()
print(end - start)

0.0


In [49]:
#creating index
index_result=db.profiles.create_index([('age',pymongo.ASCENDING)],unique=False)

In [50]:
import time
start=time.time()
#age50=collection.find({'age':{'$eq':50}})
over_35=collection.find({'age':{"$gt":55}})
end=time.time()
print(end - start)

0.0


In [2]:

###Analytics with Cassendra
from cassandra.cluster import Cluster
cluster=Cluster(['localhost'])# expects a list as parameter

In [3]:
session=cluster.connect()

In [8]:
session.execute("CREATE KEYSPACE stocks\
                    WITH replication={'class':'SimpleStrategy',\
                                        'replication_factor':'1'}")#run the query

In [9]:
session.set_keyspace('stocks')

In [12]:
session.execute("""
                    CREATE TABLE company(
                        company_id text,
                        name_latest text,
                        name_previous text,
                        PRIMARY KEY(company_id))""")

In [14]:
session.execute("""
                    CREATE TABLE indicator_by_company(
                        company_id text,
                        indicator_id text,
                        yr_2010 bigint,
                        yr_2011 bigint,
                        yr_2012 bigint,
                        yr_2013 bigint,
                        yr_2014 bigint,
                        yr_2015 bigint,
                        yr_2016 bigint,
                        PRIMARY KEY(company_id,indicator_id))""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 61))



AlreadyExists: Table 'stocks.indicator_by_company' already exists

In [16]:
#load data into table
import json
with open("companies.json") as f_in:
    companies=json.load(f_in)
    

In [17]:
type(companies)

list

In [18]:
companies[0]

{u'company_id': u'1000045',
 u'name_latest': u'Nicholas Financial Inc',
 u'names_previous': None}

In [23]:
session.execute("""
                    INSERT INTO company (company_id,name_latest,name_previous)
                    VALUES (%s,%s,%s)
                    """,
                    ("1000045", "Nicholas Financial Inc",""))

In [25]:
result_set=session.execute("SELECT * FROM company")

In [26]:
result_set.current_rows

[Row(company_id=u'1000045', name_latest=u'Nicholas Financial Inc', name_previous=u'')]

In [27]:
for company in companies:
    try:
        session.execute("""
                        
                        INSERT INTO company (company_id,name_latest,name_previous)
                        VALUES (%s,%s,%s)
                        """,
                        (company['company_id'],company['name_latest'],company['names_previous']))
    except:
        pass

In [28]:
result_set=session.execute("SELECT * FROM company")

In [ ]:
result_set

In [ ]:
import pandas as pd
income_df=pd.DataFrame(list(collection.find()))
income_df.head()
income_df['age'].describe()
income_df.mean(1)